In [1]:
import pandas as pd
from datetime import datetime
import math

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from shapely.geometry import Point
import geopandas as gpd

import matplotlib.pyplot as plt
import mplleaflet as mpl

In [2]:
# Read data from provided CSV
df = pd.read_csv('map-test.csv')

In [3]:
# Convert date strings to datetime objects
df['from'] = df['from'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df['to'] = df['to'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [4]:
# Get geodata for provided locations
geolocator = Nominatim(user_agent='Traap')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df['geodata'] = df['location'].apply(geocode)
coordinates = df['geodata'].apply(lambda x: Point(x.longitude, x.latitude))

In [5]:
# Create GeoDataFrame out of Dataframe
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=coordinates)

In [6]:
# Calculate sizes of points
size = [(ft[1]-ft[0]).days for ft in zip(gdf['from'], gdf['to'])]

In [7]:
plt.scatter(gdf['geometry'].x, gdf['geometry'].y, s=size, color='red', marker='o', alpha=0.55)
mpl.display()